# Optimal Investment with Stochastic Interest Rates and Ambiguity

The market consists of a risk-free asset, a zero-coupon bond
(both determined by a Vasicek model), and a stock. There is ambiguity about the
risk premia, the volatilities, and the correlation. The investor’s preferences display
both risk aversion and ambiguity aversion. \
The solution shows that
the investor does not hedge ambiguity but only risk, while the ambiguity only affects the speculative motives of the investor. An implementation of the optimal
investment strategy shows the impact of the different sources of ambiguity. Ambiguity aversion helps to tame the highly leveraged portfolios neglecting ambiguity
and leads to strategies that are more in line with popular investment advice.

## Introduction

Ambiguity refers to the uncertainty
that cannot be described by probabilities and is surely present in financial markets, since
model parameters are far from perfectly known and often hard to estimate./
The market offers a locally risk-free asset with a
risky short-term interest rate. In addition, there is a zero-coupon bond with deterministic
risk premium and volatility. The stock market is represented by a single risky asset with
constant risk premium, volatility, and correlation (between the stock and the short-term
interest rate as well as the bond). The investor chooses how much wealth to invest in the
different assets, which can be continuously rebalanced over time. Since there is ambiguity,
the investor does not only consider a single scenario for the risk premia, the volatilities,
and the correlation but a collection of possible scenarios without any assumptions on
which is more likely to be the correct one. Each scenario leads to a different evolution of
the investor’s wealth and the short-term interest rate. The ambiguity is represented by a set of priors where each prior represents a
specific scenario for the risk premia, the volatilities, and the correlation. The investor aims at maximizing expected utility
under the worst possible prior. The market conditions are represented by an assumption on the Sharpe ratios of the bond
and the stock under the worst-case prior; The assumption corresponds to
typical market conditions since it ensures that the bond and the stock are still profitable
under the worst-case prior. In the case without
ambiguity, the optimal investment strategy is a combination of a speculative (also called
myopic) portfolio and a portfolio hedging interest rate risk. / 
 the additional dynamic effect in the investment strategy due to
the ambiguous volatility of the short-term interest rate helps to tame the very extreme
behavior of the optimal investment strategy neglecting ambiguity. The approach to solving the optimal investment problem is based on an extension
of the martingale optimality principle.

## Investment Problem

All risk in the model is represented by a two-dimensional standard Brownian motion. Let $\left(\Omega, \mathcal{F}, P_0\right)$  be a probability space such that the process $B^{\top}=\left(B_t^1, B_t^2\right)_t$ is a twodimensional standard Brownian motion under $P_0$. The process B consists of the risk factors that drive the
dynamics of the available investment opportunities in the market. The market offers a risk-free asset, determined by the short-term interest rate. The
(locally) risk-free asset satisfies the price dynamics

$d P_t^0=P_t^0 r_t d t$

In [2]:
#Import libraries
import sympy as smp

In [3]:
#define symbols and function
P_t0 ,r_t, t, rbar, sigma_r, dB_t1, k = smp.symbols('P_t0 r_t t rbar sigma_r dB_t1 k', real=True) 
dP_t0 = smp.symbols('dP_t0', cls=smp.Function)
dr_t = smp.symbols('dr_t', cls=smp.Function)


In [4]:
dP_t0 = smp.diff(P_t0*r_t,t)


where the process $r = (r_t)_t$ denotes the short-term interest rate. The short-term interest
rate is stochastic and evolves as in the (classical) Vasicek model:\
$d r_t=\kappa\left(\bar{r}-r_t\right) d t+\sigma_r d B_t^1$

In [5]:
dr_t = smp.diff(k*(rbar-r_t),t) + sigma_r * dB_t1


the short-term interest rate is a mean-reverting process with a constant volatility. The parameters $\kappa, \bar{r}, \sigma_r \in \mathbb{R}$, where $\sigma_r>0$ where $σ_r > 0$, represent the mean reversion speed and level
and the volatility, respectively. In addition to the risk-free asset, the investor can invest in a zero-coupon bond, which
is risky. According to the Vasicek model, a zero-coupon bond with maturity $\bar{T}$  is priced
such that its price evolves as \
 $d P_t^B=P_t^B\left(\left(r_t+b(\bar{T}-t) \sigma_r \lambda_B\right) d t-b(\bar{T}-t) \sigma_r d B_t^1\right)$  
The function b  which is defined by \
 $b(\bar{T}-t):=\frac{1}{\kappa}\left(1-e^{-\kappa(\bar{T}-t)}\right)$

In [6]:
dP_tB ,P_tB, b, Tbar, lambda_B = smp.symbols('dP_tB P_tB b Tbar lambda_B', real=True) 



In [7]:
dP_tB = P_tB*((smp.diff(r_t+b*(Tbar - t)*sigma_r*lambda_B,t))-b*(Tbar - t)*sigma_r *dB_t1)
smp.Eq((b*(Tbar - t)), (1/k*(1-smp.exp(-k*(Tbar - t)))))

Eq(b*(Tbar - t), (1 - exp(-k*(Tbar - t)))/k)


determines the volatility of the bond in terms of the volatility of the short-term interest
rate. It is an increasing function; thus, long-term bonds are riskier than short-term
bonds.  The Sharpe ratio
of the bond is represented by $\lambda_b$  which is assumed to be constant in the Vasicek
model. This assumption makes the model tractable but is (of course) very unrealistic.
Introducing ambiguity to the model in the following helps to overcome this trade-off.
Moreover, the investor can invest in the stock market. The stock market is represented 
by a single risky asset with price dynamics

$d P_t^S=P_t^S\left(\left(r_t+\lambda_S\right) d t+\sigma_S \rho d B_t^1+\sigma_S \sqrt{1-\rho^2} d B_t^2\right)$

In [8]:
sigma_S, lambda_S, dP_tS,rho, dB_t2, B_t2, P_tS = smp.symbols('sigma_S lambda_S dP_tS rho dB_t2 B_t2 P_tS', real=True) 
dP_tS = P_tS * (smp.diff(r_t+lambda_S, t) + sigma_S * rho * dB_t1 + sigma_S*smp.sqrt(1-rho**2) * dB_t2)



The stock is driven by both risk factors–so the stock price is driven by some additional
noise, while the first noise term allows for some correlation between the short-term interest
rate and the stock. The correlation and the volatility are represented by the parameters $\rho$  and $\sigma_S$ 

The investor participates in the market by choosing a dynamic investment strategy,
which can be continuously rebalanced over time. An investment strategy is an $\mathbb{F}{-}$ adapted process $\pi^{\top}=\left(\pi_t^B, \pi_t^S\right)_t$, where $\pi_t^B$ and $\pi_t^S$ are the fractions of wealth invested in the bond and the stock, respectively, at time $t$. The remaining wealth is invested in the risk-free asset. For an investment strategy $\pi$, the investor's wealth, given by the process $W^\pi=\left(W_t^\pi\right)_t$, evolves as

$d W_t^\pi=W_t^\pi\left(\left(r_t+\pi_t^{\top} \lambda(t)\right) d t+\pi_t^{\top} \sigma(t) d B_t\right)$

In [9]:
dB_t,sigmat,dW_tPi, W_tPi,lambdat,pi_tT = smp.symbols('dB_t sigmat dW_tPi W_tPi lambdat pi_tT', real=True) 
dW_tPi = W_tPi * (smp.diff(r_t + pi_tT * lambdat,t) + pi_tT * sigmat* dB_t)


starting from the investor’s initial wealth, denoted by $W_0$, where

$\lambda(t):=\left(\begin{array}{c}b(\bar{T}-t) \sigma_r \lambda_B \\ \lambda_S\end{array}\right), \quad \sigma(t):=\left(\begin{array}{cc}-b(\bar{T}-t) \sigma_r & 0 \\ \sigma_S \rho & \sigma_S \sqrt{1-\rho^2}\end{array}\right)$

In [10]:
lambdat = smp.Matrix([b*(Tbar-t)*sigma_r*lambda_B,lambda_S])
sigmat = smp.Matrix([[-b*(Tbar-t)*sigma_r, sigma_S*rho],[0,sigma_S*smp.sqrt(1-rho**2)]])
sigmat

Matrix([
[-b*sigma_r*(Tbar - t),              rho*sigma_S],
[                    0, sigma_S*sqrt(1 - rho**2)]])

Since there is ambiguity, the investor considers several scenarios for the risk premia, the volatilities, and the correlation of the available assets instead of a fixed parameter combination. A fixed parameter combination is given by $\left(\lambda_B, \lambda_S, \sigma_r, \sigma_S, \rho\right)$ in the setting above. The space of possible scenarios is denoted by $\Theta$, which consists of all $\mathbb{F}$-adapted processes $\theta=\left(\lambda_t^B, \lambda_t^S, \sigma_t^r, \sigma_t^S, \rho_t\right)_t$ satisfying for all $t$
$$
\underline{\lambda}_B(t) \leq \lambda_t^B \leq \bar{\lambda}_B(t), \quad \underline{\lambda}_S \leq \lambda_t^S \leq \bar{\lambda}_S, \quad \underline{\sigma}_r \leq \sigma_t^r \leq \bar{\sigma}_r, \quad \underline{\sigma}_S \leq \sigma_t^S \leq \bar{\sigma}_S, \quad \underline{\rho} \leq \rho_t \leq \bar{\rho}
$$

 The extreme values for the risk premium on the stock, both volatilities, and the
correlation are given exogenously. The extreme values for the risk premium on the bond
are determined endogenously by the volatility of the short-term interest rat . The reason
is that an arbitrage-free term structure in the presence of ambiguous volatility requires
an ambiguous process in the risk premium of a zero-coupon bond. In a Vasicek model with ambiguous volatility, the risk premium on a zero-coupon bond with maturity $\bar{T}$ is $b(\bar{T}-t) \lambda_t^B$, where the ambiguity in the risk premium, represented by $\lambda_t^B$, is bounded by
$$
\begin{aligned}
& \underline{\lambda}_B(t)=e^{-2 \kappa t} \lambda_0^B+\frac{\underline{\sigma}_r^2}{2 \kappa}\left(1-e^{-2 \kappa t}\right) \\
& \bar{\lambda}_B(t)=e^{-2 \kappa t} \lambda_0^B+\frac{\bar{\sigma}_r^2}{2 \kappa}\left(1-e^{-2 \kappa t}\right)
\end{aligned}
$$\


In [11]:
sigmaBar_r, lambdaBar_Bt, underSigma_r,lambda_0B, underLambda_Bt, barUnderLambda_Bt = smp.symbols('sigmaBar_r lambdaBar_Bt underSigma_r lambda_0B underLambda_Bt, barUnderLambda_Bt', real=True) 

In [12]:
underLambda_Bt = smp.exp(-2*k*t)*lambda_0B + (underSigma_r**2 / (2*k) ) * (1-smp.exp(-2*k*t))
lambdaBar_Bt = smp.exp(-2*k*t)*lambda_0B + (sigmaBar_r**2 / (2*k)) * (1-smp.exp(-2*k*t))
lambdaBar_Bt

lambda_0B*exp(-2*k*t) + sigmaBar_r**2*(1 - exp(-2*k*t))/(2*k)

The extreme values for the ambiguous risk premium on the bond are
essentially determined by the extreme values for the ambiguous variance of the shortterm interest rate. Compared to the original results $\lambda_0^B$ is nonzero, since otherwise, the initial value of the risk premium on the bond is zero. Consequently, the investor’s wealth and the short-term interest rate have a different
evolution in each possible scenario. For an investment strategy π and a scenario θ the investor's wealth and the short-term interest rate, now denoted by $W^{\pi, \theta}=\left(W_t^{\pi, \theta}\right)_t$ and $r^\theta=\left(r_t^\theta\right)_t$, respectively, evolve as
$$
\begin{aligned}
d W_t^{\pi, \theta} & =W_t^{\pi, \theta}\left(\left(r_t^\theta+\pi_t^{\top} \lambda_t^\theta\right) d t+\pi_t^{\top} \sigma_t^\theta d B_t\right) \\
d r_t^\theta & =\kappa\left(\bar{r}-r_t\right) d t+\nu_t^\theta d B_t
\end{aligned}
$$


In [20]:
v_ttheta, dr_ttheta, sigma_ttheta, lambda_ttheta, W_tpitheta,dW_tpitheta, r_ttheta,pi_tT = smp.symbols('v_ttheta dr_ttheta sigma_ttheta lambda_ttheta W_tpitheta dW_tpitheta r_ttheta pi_tT', real=True) 
dW_tpitheta = W_tpitheta * (smp.diff(r_ttheta + pi_tT* lambda_ttheta,t) + pi_tT*sigma_ttheta * dB_t)
dr_ttheta = k * smp.diff(rbar - r_t,t) + v_ttheta * dB_t
dr_ttheta

dB_t*v_ttheta


starting from the investor's initial wealth and the initial short-term interest rate, denoted by $W_0$ and $r_0$, respectively, where
$$
\lambda_t^\theta:=\left(\begin{array}{c}
b(\bar{T}-t) \lambda_t^B \\
\lambda_t^S
\end{array}\right), \quad \sigma_t^\theta:=\left(\begin{array}{cc}
-b(\bar{T}-t) \sigma_t^r & 0 \\
\sigma_t^S \rho_t & \sigma_t^S \sqrt{1-\rho_t^2}
\end{array}\right), \quad \nu_t^\theta:=\left(\begin{array}{c}
\sigma_t^r \\
0
\end{array}\right)
$$


In [25]:
lambda_ttheta,lambda_tB, lambda_tS, sigma_tr, sigma_tS, rho_t = smp.symbols('lambda_ttheta lambda_tB lambda_tS sigma_tr sigma_tS rho_t', real=True) 
lambda_ttheta = smp.Matrix([b*(Tbar - t)* lambda_tB, lambda_S])
sigma_ttheta = smp.Matrix([[-b*(Tbar - t)*sigma_tr, sigma_tS*rho_t],[0, sigma_tS * smp.sqrt(1-rho_t**2)]])
v_ttheta = smp.Matrix([sigma_tr,0])


Matrix([
[sigma_tr],
[       0]])


The investor is still restricted to choose a sufficiently regular investment strategy. The space of admissible investment strategies is denoted by $\Pi$, which consists of all $\mathbb{F}$-adapted processes $\pi$ that are bounded, ensuring that the wealth process is sufficiently regular for all $\theta \in \Theta$, that is, in each possible scenario.

The ambiguity is represented by a set of priors, where each prior represents a different scenario. Similar to Epstein and Ji [13, 14], one can construct a prior related to each scenario for the evolution of the wealth process and the short-term interest rate. For each investment strategy $\pi \in \Pi$ and each possible scenario $\theta \in \Theta$, define the prior
$$
P_{\pi, \theta}:=P_0 \circ\left(W^{\pi, \theta}, r^\theta\right)^{-1}
$$
> This notation means that you take the output of inside the parethesis 
 and use it as the input of $P_0$


Then the set of priors, related to the investment strategy $\pi \in \Pi$, is defined by
$$
\mathcal{P}_\pi:=\left\{P_{\pi, \theta} \mid \theta \in \Theta\right\}
$$
Hence, the canonical process, denoted by $(W, r)=\left(W_t, r_t\right)_t$, which represents the wealth and the short-term interest rate, evolves according to a different scenario under each prior.

The investor has preferences about terminal wealth from investing, displaying both risk and ambiguity aversion. As in similar studies, the investor cares about terminal wealth. The preferences are represented by expected utility with a constant relative risk aversion (CRRA) utility function, representing the investor's risk aversion. The attitude towards ambiguity is incorporated by maxmin expected utility in the spirit of Gilboa and Schmeidler [18]: the investor ranks investment strategies according to their expected utility from terminal wealth under the worst-case scenario among the relevant priors. 

## Optimal Investment

In order to solve the optimal investment problem in closed form, the extreme values for
the ambiguous quantities are supposed to obey the following market conditions. The
assumption corresponds to typical market conditions

### Assumption 3.1
$-\frac{1}{\rho} \frac{\lambda_S}{\bar{\sigma}_S} \geq \frac{\lambda_B(t)}{\bar{\sigma}_r} \geq-\underline{\rho} \frac{\lambda_S}{\bar{\sigma}_S}$.

Without the assumption, it is not possible to obtain a sufficiently smooth value function for the optimal investment problem, which is essential for solving dynamic asset allocation problems in closed form by dynamic programming as well as applying the martingale optimality principle in Section A. The time-varying extreme values for the ambiguity in the risk premium on the bond can lead to switching worst-case parameters over time, which leads in turn to a nondifferentiable value function. Since the implementation of the optimal investment strategy in Section 4 shows that Assumption 3.1 corresponds to typical market situations, the focus of the analysis lies on a region of the state space where the investment problem offers a closed-form solution, providing insights into the structure of the optimal investment strategy.

Form an economic point of view, Assumption 3.1 ensures that the bond and the stock are both profitable under (what turns out to be) the worst-case prior. The assumption is formulated in terms of the worst-case Sharpe ratios of the bond, $\underline{\lambda}_B(t) / \bar{\sigma}_r$, and the stock, $\underline{\lambda}_S / \bar{\sigma}_S$, and yields a long position in the bond and the stock in the speculative, i.e., meanvariance optimal, part of the optimal investment strategy, which can be deduced from Theorem 3.2 and the succeeding discussion. Consequently, the assumption circumvents undesirable strategies with short positions in the risky assets. Hence, the advantage of Assumption 3.1 is that it rules out too extreme scenarios for the ambiguous quantities, since having too much ambiguity might lead to very extreme positions.

Under the market conditions stated in Assumption 3.1, the optimal investment problem can be solved in closed form.

Theorem 3.2. Let Assumption 3.1 be satisfied. Then the optimal investment strategy is given $b y \hat{\pi}^{\top}=\left(\hat{\pi}_t^B, \hat{\pi}_t^S\right)_t$, where
$$
\begin{aligned}
\hat{\pi}_t^B & :=\frac{1}{\gamma} \frac{1}{1-\underline{\rho}^2} \frac{1}{b(\bar{T}-t) \bar{\sigma}_r}\left(\frac{\underline{\lambda}_B(t)}{\bar{\sigma}_r}+\underline{\rho} \underline{\underline{\lambda}}_S\right)+\frac{\gamma-1}{\gamma} \frac{b(T-t)}{b(\bar{T}-t)} \\
\hat{\pi}_t^S & :=\frac{1}{\gamma} \frac{1}{1-\underline{\rho}^2} \frac{1}{\bar{\sigma}_S}\left(\frac{\underline{\lambda}_S}{\bar{\sigma}_S}+\underline{\rho} \frac{\underline{\lambda}_B(t)}{\bar{\sigma}_r}\right)
\end{aligned}
$$


In [34]:
T,sigmaBar_S, underlambda_S, pihat_tB, gamma, underrho, pihat_tS = smp.symbols('T sigmaBar_S underlambda_S pihat_tB gamma underrho pihat_tS', real=True) 
pihat_tB = (1/gamma) * (1/(1-underrho**2)) * (1/(b*(Tbar - t)*sigmaBar_r)) * ((underLambda_Bt / sigmaBar_r) + underrho * (underlambda_S / sigmaBar_S)) + (gamma-1*b*(T-t)/(gamma*b*(Tbar-t)))
pihat_tS = (1/(gamma*1-underrho**2*sigmaBar_S)) * ((underlambda_S/sigmaBar_S)+(underrho*(underLambda_Bt/sigmaBar_r)))
pihat_tS

(underrho*(lambda_0B*exp(-2*k*t) + underSigma_r**2*(1 - exp(-2*k*t))/(2*k))/sigmaBar_r + underlambda_S/sigmaBar_S)/(gamma - sigmaBar_S*underrho**2)

the worst-case prior for $\hat{\pi}$ is given by $P_{\hat{\pi}, \hat{\theta}}$, where
$$
\hat{\theta}=\left(\underline{\lambda}_B(t), \underline{\lambda}_S, \bar{\sigma}_r, \bar{\sigma}_S, \underline{\rho}\right)_t
$$
and the value of the optimal investment problem is given by
$$
\sup _{\pi \in \Pi} \inf _{P \in \mathcal{P}_\pi} \mathbb{E}_P\left[u\left(W_T\right)\right]=V\left(0, W_0, r_0\right)
$$
where the value function $V:[0, T] \times \mathbb{R}_{+} \times \mathbb{R} \rightarrow \mathbb{R}$ is defined by
$$
V(t, W, r):=\exp \left((1-\gamma)\left(a_0(t)+a_1(t) r\right)\right) \frac{W^{1-\gamma}}{1-\gamma}
$$


In [36]:
W,r,VtWr, a_0, a_1t, a_0t = smp.symbols('W r VtWr a_0 a_1t a_0t ', real=True) 
VtWr = smp.exp((1-gamma)*(a_0t + a_1t*r)) * ((W**(1-gamma))/(1-gamma))
VtWr

W**(1 - gamma)*exp((1 - gamma)*(a_0t + a_1t*r))/(1 - gamma)

and the functions $a_0, a_1:[0, T] \rightarrow \mathbb{R}$ are defined by
$$
\begin{aligned}
a_0(t):= & \frac{1}{2} \frac{1}{\gamma} \frac{1}{1-\underline{\rho}^2} \int_t^T\left(\frac{\hat{\lambda}_B(u)^2}{\bar{\sigma}_r^2}+2 \underline{\rho} \frac{\lambda_B(u) \underline{\lambda}_S}{\bar{\sigma}_r \bar{\sigma}_S}+\frac{\lambda_S^2}{\bar{\sigma}_S^2}\right) d u+\kappa \bar{r} \int_t^T b(T-u) d u \\
& +\frac{\gamma-1}{\gamma} \int_t^T \underline{\lambda}_B(u) b(T-u) d u-\frac{1}{2} \frac{\gamma-1}{\gamma} \bar{\sigma}_r^2 \int_t^T b(T-u)^2 d u
\end{aligned}
$$
and $a_1(t):=b(T-t)$, respectively.

In [44]:
underLambda_Bu, lambda_Bu, u,lambdahat_Bu = smp.symbols('underLambda_Bu lambda_Bu u lambdahat_Bu', real=True) 
expr1 = smp.integrate((lambdahat_Bu / sigmaBar_r**2) + 2*underSigma_r * ((lambda_Bu*underlambda_S)/(sigmaBar_r*sigmaBar_S)) + (lambda_S**2/sigmaBar_S**2), (u,t,T))
expr2 = smp.integrate(underLambda_Bu*b*(T-t), (u,t,T))
a_0t = 1/(2*gamma*(1-underrho**2)) * expr1 + k*rbar * smp.integrate(b*(T-u),(u,t,T)) + ((gamma - 1)/gamma) * expr2 * smp.integrate( b*(T-t), (u,t,T))
a_1t = b*(T-t)
a_0t

k*rbar*(T**2*b/2 - T*b*t + b*t**2/2) + (gamma - 1)*(T*b*(T - t) - b*t*(T - t))*(T*b*underLambda_Bu*(T - t) - b*t*underLambda_Bu*(T - t))/gamma + (T*(2*lambda_Bu*underSigma_r*underlambda_S/(sigmaBar_S*sigmaBar_r) + lambda_S**2/sigmaBar_S**2 + lambdahat_Bu/sigmaBar_r**2) - t*(2*lambda_Bu*underSigma_r*underlambda_S/(sigmaBar_S*sigmaBar_r) + lambda_S**2/sigmaBar_S**2 + lambdahat_Bu/sigmaBar_r**2))/(2*gamma*(1 - underrho**2))

Depending on the level of risk aversion, the investor allocates wealth between two portfolios-a speculative portfolio and a portfolio that hedges interest rate risk. As in the case without ambiguity, Theorem 3.2 shows that the optimal portfolio $\hat{\pi}$ consists of two funds:
$$
\hat{\pi}_t=\frac{1}{\gamma} \pi_t^{\text {myopic }}+\frac{\gamma-1}{\gamma} \pi_t^{\text {hedge }}
$$


In [46]:
pihat_t,pi_tmyopic,pi_thedge = smp.symbols('pihat_t pi_tmyopic pi_thedge', real=True) 

pihat_t = (1/gamma) * pi_tmyopic + (gamma - 1) / gamma + pi_thedge
pihat_t

pi_thedge + pi_tmyopic/gamma + (gamma - 1)/gamma

where the investor's level of risk aversion, $\gamma$, determines how wealth is allocated between both funds. The portfolio $\pi^{\text {myopic }}=\left(\pi_t^{\text {myopic }}\right)_t$ is given by
$$
\pi_t^{\text {myopic }}:=\frac{1}{1-\underline{\rho}^2}\left(\begin{array}{c}
\frac{1}{b(\bar{T}-t) \bar{\sigma}_r}\left(\frac{\underline{\lambda}_B(t)}{\bar{\sigma}_r}+\underline{\rho} \underline{\frac{\lambda_S}{}}\right) \\
\frac{1}{\bar{\sigma}_S}\left(\frac{\lambda_S}{\bar{\sigma}_S}+\underline{\rho} \frac{\underline{\lambda}_B(t)}{\bar{\sigma}_r}\right)
\end{array}\right)
$$


In [51]:
ex1 = 1/(b*(Tbar - t)*sigmaBar_r) * (underLambda_Bt / sigmaBar_r + underrho * underlambda_S )
ex2 = (1/sigmaBar_S) * ((underlambda_S / sigmaBar_S) + underrho * (underLambda_Bt / sigmaBar_r))
pi_tmyopic = (1/(1-underrho**2)* smp.Matrix([ex1,ex2]))
pi_tmyopic

Matrix([
[(underlambda_S*underrho + (lambda_0B*exp(-2*k*t) + underSigma_r**2*(1 - exp(-2*k*t))/(2*k))/sigmaBar_r)/(b*sigmaBar_r*(1 - underrho**2)*(Tbar - t))],
[  (underrho*(lambda_0B*exp(-2*k*t) + underSigma_r**2*(1 - exp(-2*k*t))/(2*k))/sigmaBar_r + underlambda_S/sigmaBar_S)/(sigmaBar_S*(1 - underrho**2))]])

and represents the speculative part, since it corresponds to the classical mean-variance optimal investment strategy, which is typically referred to as the myopic portfolio. The portfolio $\pi^{\text {hedge }}=\left(\pi_t^{\text {hedge }}\right)_t$ is given by
$$
\pi_t^{\text {hedge }}:=\left(\begin{array}{c}
\frac{b(T-t)}{b(\bar{T}-t)} \\
0
\end{array}\right)
$$

In [52]:
pi_thedge = smp.Matrix([((b*(T-t))/(b*(Tbar-t))),0])
pi_thedge

Matrix([
[(T - t)/(Tbar - t)],
[                 0]])

which is referred to as the hedge portfolio, since it hedges interest rate risk using only the bond, as in the classical situation without ambiguity.

Therefore, the investor does not hedge ambiguity but only risk. Introducing ambiguity to the model does not lead to any additional funds compared to the case without ambiguity. This is in line with other studies on optimal investment with ambiguity and maxmin preferences $[4,27]$ but in contrast to the results of Flor and Larsen [15] and Maenhout [30], where ambiguity leads to an additional hedge term. The reason is that the preferences of the investor in the present study do not include any penalty term, which is added to the objective of the investor by Flor and Larsen [15] and Maenhout [30]. Therefore, the absence of additional hedge terms is due to the maxmin preferences of the investor, which do not create any demand for hedging ambiguity.

In particular, ambiguity only affects the speculative part of the investment, but it has no effect on hedging interest rate risk. The effect of the ambiguity is represented by the worst-case scenario $\hat{\theta}=\left(\underline{\lambda}_B(t), \underline{\lambda}_S, \bar{\sigma}_r, \bar{\sigma}_S, \underline{\rho}\right)_t$. So the investor considers the lowest possible risk premia, the highest possible volatilities, and the lowest possible correlation as the worst-case scenario among all relevant priors. The worst-case scenario only appears in the speculative portfolio, $\pi^{\text {myopic }}$. The hedge portfolio, $\pi^{\text {hedge }}$, is unaffected by the worst-case scenario. At first sight, this is surprising since the hedge portfolio is typically determined by the diffusion terms of the wealth and the state variable, respectivelywhich are ambiguous in this case. In principle (when deriving a candidate for the optimal investment strategy under a fixed scenario), the hedge portfolio in the optimal investment strategy depends on the (ambiguous) volatility of the short-term interest rate, but in the end, it always vanishes (no matter which scenario is considered by the investor). Thus, the hedge portfolio is only influenced by the function $b$, which scales the ambiguous volatility of the bond.

In contrast to the exogenous sources of ambiguity, the endogenous ambiguity leads to an additional dynamic effect in the investment strategy. As in the case without ambiguity the investment strategy changes if the time horizon of the investor changes, which is an interesting feature, since it is intuitively reasonable but not present in classical dynamic asset allocation models. More specifically, if there is no ambiguity, the optimal investment strategy is given by $\tilde{\pi}^{\top}=\left(\tilde{\pi}_t^B, \tilde{\pi}_t^S\right)_t$, where that is, the investor only considers the (constant) scenario $\tilde{\theta}=\left(\lambda_0^B, \lambda_S, \sigma_r, \sigma_S, \rho\right)_t($ similar to the beginning of Section 2). Since everything else is constant, the dynamic behavior of the investment strategy is completely determined by the function $b$, which changes according to the time to maturity of the bond and how much time is left until the investor's time horizon. In the presence of ambiguity, the fixed parameters are replaced by their respective worst-case values (compare the investment strategies $\tilde{\pi}$ from above and $\hat{\pi}$ from Theorem 3.2). The worst-case values related to the exogenous ambiguity do not create any dynamic behavior, but the worst-case value for the ambiguous part in the risk premium on the bond, $\underline{\lambda}_B(t)$, changes over time, which creates an additional dynamic effect in addition to the (classical) effect induced by the function $b$.

> all the text and formula are taken from the paper: Optimal Investment with Stochastic Interest Rates and Ambiguity by Julian Hölzermann
link: https://arxiv.org/abs/2306.13343